## Train-Test Split

In [19]:
import os
import pandas as pd

In [37]:
images_list = os.listdir('Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/images/train')
labels_list = os.listdir('Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/labels/train')

In [38]:
images_df=[]
for image in images_list:
    if image.replace('.jpeg', '.txt') in labels_list:
        images_df.append({
            'subject': image.split('_')[1],
            'image': image,
            'has_bb': 1
        })
    else:
        images_df.append({
            'subject': image.split('_')[1],
            'image': image,
            'has_bb': 0
        })

images_df = pd.DataFrame(images_df)

In [39]:
images_df

,subject,image,has_bb
0,3930,image_3930_7.jpeg,0
1,5736,image_5736_5.jpeg,1
2,5544,image_5544_3.jpeg,0
3,5593,image_5593_18.jpeg,1
4,6048,image_6048_9.jpeg,1
...,...,...,...
2495,5915,image_5915_10.jpeg,0
2496,3034,image_3034_8.jpeg,0
2497,5690,image_5690_11.jpeg,0
2498,5680,image_5680_9.jpeg,0


In [49]:
from sklearn.model_selection import StratifiedGroupKFold

stratified_group_k_fold = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

X = images_df['image']
y = images_df['has_bb']
groups = images_df['subject']

for i, (train_index, test_index) in enumerate(stratified_group_k_fold.split(X, y, groups)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"         group={groups[train_index]}")
    print(f"  Test:  index={test_index}")
    print(f"         group={groups[test_index]}")
    break

Fold 0:
  Train: index=[   0    1    2 ... 2497 2498 2499]
         group=0       3930
1       5736
2       5544
3       5593
5       5700
        ... 
2495    5915
2496    3034
2497    5690
2498    5680
2499    2413
Name: subject, Length: 1982, dtype: object
  Test:  index=[   4   11   16   18   19   21   27   33   41   43   46   47   50   60   64   68   72   77   86   87   97  101  102  108  109  112  114  122  124  126  132  134  156  157  160  161  165  168  181  182  186  192  193  199  203  209  211  218  220  224  243  244  245  250  252  260  262  279  284  288  289  290  296
  301  304  307  308  310  318  327  341  342  343  344  345  349  350  351  356  361  371  376  378  384  393  394  395  399  400  401  410  411  413  416  424  426  427  436  441  442  447  459  461  468  476  480  488  493  495  506  513  530  532  535  544  557  559  560  561  562  571  572  576  577  580  581
  583  584  592  593  596  598  603  605  612  627  635  655  658  666  668  671  672  673  6

In [50]:
train_index

array([   0,    1,    2, ..., 2497, 2498, 2499])

In [51]:
test_index

array([   4,   11,   16,   18,   19,   21,   27,   33,   41,   43,   46,   47,   50,   60,   64,   68,   72,   77,   86,   87,   97,  101,  102,  108,  109,  112,  114,  122,  124,  126,  132,  134,  156,  157,  160,  161,  165,  168,  181,  182,  186,  192,  193,  199,  203,  209,  211,  218,  220,  224,  243,  244,
        245,  250,  252,  260,  262,  279,  284,  288,  289,  290,  296,  301,  304,  307,  308,  310,  318,  327,  341,  342,  343,  344,  345,  349,  350,  351,  356,  361,  371,  376,  378,  384,  393,  394,  395,  399,  400,  401,  410,  411,  413,  416,  424,  426,  427,  436,  441,  442,  447,  459,  461,  468,
        476,  480,  488,  493,  495,  506,  513,  530,  532,  535,  544,  557,  559,  560,  561,  562,  571,  572,  576,  577,  580,  581,  583,  584,  592,  593,  596,  598,  603,  605,  612,  627,  635,  655,  658,  666,  668,  671,  672,  673,  677,  683,  689,  702,  703,  704,  708,  712,  718,  726,  728,  736,
        743,  745,  749,  753,  762,  763, 

In [52]:
images_df['partition'] = None
images_df.loc[train_index, 'partition'] = 'train'
images_df.loc[test_index, 'partition'] = 'val'

In [53]:
images_df

,subject,image,has_bb,partition
0,3930,image_3930_7.jpeg,0,train
1,5736,image_5736_5.jpeg,1,train
2,5544,image_5544_3.jpeg,0,train
3,5593,image_5593_18.jpeg,1,train
4,6048,image_6048_9.jpeg,1,val
...,...,...,...,...
2495,5915,image_5915_10.jpeg,0,train
2496,3034,image_3034_8.jpeg,0,train
2497,5690,image_5690_11.jpeg,0,train
2498,5680,image_5680_9.jpeg,0,train


In [55]:
images_df[images_df.has_bb == 0]['partition'].value_counts()

partition
train    1602
val       403
Name: count, dtype: int64

In [58]:
# Moving the images to the corresponding folders
import shutil

val_images = images_df[images_df['partition'] == 'val']

#Creating the folders
os.makedirs('Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/images/val', exist_ok=True)
os.makedirs('Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/labels/val', exist_ok=True)

for i,row in val_images.iterrows():
    # Moving the images
    image = row['image']
    shutil.move(f'Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/images/train/{image}', f'Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/images/val/{image}')
    
    if row['has_bb'] == 1:
        # Moving the labels
        shutil.move(f'Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/labels/train/{image.replace(".jpeg", ".txt")}', f'Prostate_Cancer_TFM/Bounding_Boxes/Prostate_Data/labels/val/{image.replace(".jpeg", ".txt")}')
    

## Training

In [1]:
from ultralytics import YOLO

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


In [3]:
model = YOLO('yolov8l.pt')

In [4]:
!nvidia-smi

Tue Mar  5 11:03:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   32C    P0    36W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# Train the model

results = model.train(data='Prostate_Cancer_TFM/Bounding_Boxes/prostate.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.1.23 🚀 Python-3.10.13 torch-2.2.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=Prostate_Cancer_TFM/Bounding_Boxes/prostate.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sh

train: Scanning /clinicfs/userhomes/jaalzate/Prostate_Cancer_TFM/Bounding_Boxes/datasets/Prostate_Data/labels/train.cache... 380 images, 1602 backgrounds, 0 corrupt: 100%|██████████| 1982/1982 [00:00<?, ?it/s]
/home/jaalzate/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 7, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /clinicfs/userhomes/jaalzate/Prostate_Cancer_TFM/Bounding_Boxes/datasets/Prostate_Data/labels/val.cache... 115 images, 403 backgrounds, 0 corrupt: 100%|██████████| 518/518 [00:00<?, ?it/s]
/home/jaalzate/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: T

Plotting labels to runs/detect/train13/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train13
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      10.8G      3.164      7.937      2.822          4        640: 100%|██████████| 124/124 [00:40<00:00,  3.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  3.66it/s]

                   all        518        121          0          0          0          0


...

    100/100      11.2G      2.148      4.015      2.037          1        640: 100%|██████████| 124/124 [00:35<00:00,  3.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.70it/s]

                   all        518        121    0.00393      0.545     0.0384     0.0127



100 epochs completed in 1.141 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 87.6MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 87.6MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics YOLOv8.1.23 🚀 Python-3.10.13 torch-2.2.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.74it/s]


                   all        518        121    0.00532      0.521     0.0406     0.0135
Speed: 0.2ms preprocess, 4.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train13
